# Load Competition Dataset

Competition dataset located in "/kaggle/input"; This path defined by Kaggle to access the competition file. We will list two files from this path as input files.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path=os.path.join(dirname, filename)
        if 'train' in path:
            __training_path=path
        elif 'test' in path:
            __test_path=path

## Check training and test path

In [2]:
#loaded files
print(f'Training path:{__training_path}\nTest path:{__test_path}')

Training path:/kaggle/input/house-prices-advanced-regression-techniques/train.csv
Test path:/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
# Kaggle Environment Prepration
#update kaggle env
import sys
#you may update the environment that allow you to run the whole code
!{sys.executable} -m pip install --upgrade scikit-learn=="0.24.2"

     |████████████████████████████████| 22.3 MB 5.2 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.3 which is incompatible.
hypertools 0.7.0 requires scikit-learn!=0.22,<0.24,>=0.19.1, but you have scikit-learn 0.24.2 which is incompatible.


In [4]:
#record this information if you need to run the Kernel internally
import sklearn; sklearn.show_versions() 


System:
    python: 3.7.10 | packaged by conda-forge | (default, Sep 13 2021, 19:43:44)  [GCC 9.4.0]
executable: /opt/conda/bin/python
   machine: Linux-5.10.68+-x86_64-with-debian-buster-sid

Python dependencies:
          pip: 21.2.4
   setuptools: 58.0.4
      sklearn: 0.24.2
        numpy: 1.19.5
        scipy: 1.7.1
       Cython: 0.29.24
       pandas: 1.3.3
   matplotlib: 3.4.3
       joblib: 1.0.1
threadpoolctl: 2.2.0

Built with OpenMP: True


# Input Dataset

In [5]:
def __load__data(__training_path, __test_path, concat=False):
	"""load data as input dataset
	params: __training_path: the training path of input dataset
	params: __test_path: the path of test dataset
	params: if it is True, then it will concatinate the training and test dataset as output
	returns: generate final loaded dataset as dataset, input and test
	"""
	# LOAD DATA
	import pandas as pd
	__train_dataset = pd.read_csv(__training_path, delimiter=',')
	__test_dataset = pd.read_csv(__test_path, delimiter=',')
	return __train_dataset, __test_dataset
__train_dataset, __test_dataset = __load__data(__training_path, __test_path, concat=True)
__train_dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# STORE SUBMISSION RELEVANT COLUMNS
__test_dataset_submission_columns = __test_dataset['Id']

In [7]:
# DISCARD IRRELEVANT COLUMNS
__train_dataset.drop(['Id'], axis=1, inplace=True)
__test_dataset.drop(['Id'], axis=1, inplace=True)

## Remove Missing Data in Numerical Columns

In the given input dataset there are <b>37 columns </b> with  missing data as follows:

MSSubClass, BsmtFinSF2, Fireplaces, FullBath, LotArea, GarageYrBlt, GarageCars, BsmtHalfBath, BsmtFinSF1, OverallCond, MiscVal, TotRmsAbvGrd, YrSold, KitchenAbvGr, HalfBath, YearBuilt, 1stFlrSF, GarageArea, MoSold, LotFrontage, BsmtFullBath, YearRemodAdd, ScreenPorch, WoodDeckSF, GrLivArea, SalePrice, OpenPorchSF, BedroomAbvGr, 3SsnPorch, BsmtUnfSF, EnclosedPorch, PoolArea, LowQualFinSF, MasVnrArea, 2ndFlrSF, OverallQual, TotalBsmtSF

The following code removes the missing values from those columns. We use average value (median) of each column to replace the null values.

In [8]:
# PREPROCESSING-1
from sklearn.impute import SimpleImputer
import numpy as np
_NUMERIC_COLS_WITH_MISSING_VALUES = ['MSSubClass', 'BsmtFinSF2', 'Fireplaces', 'FullBath', 'LotArea', 'GarageYrBlt', 'GarageCars', 'BsmtHalfBath', 'BsmtFinSF1', 'OverallCond', 'MiscVal', 'TotRmsAbvGrd', 'YrSold', 'KitchenAbvGr', 'HalfBath', 'YearBuilt', '1stFlrSF', 'GarageArea', 'MoSold', 'LotFrontage', 'BsmtFullBath', 'YearRemodAdd', 'ScreenPorch', 'WoodDeckSF', 'GrLivArea', 'SalePrice', 'OpenPorchSF', 'BedroomAbvGr', '3SsnPorch', 'BsmtUnfSF', 'EnclosedPorch', 'PoolArea', 'LowQualFinSF', 'MasVnrArea', '2ndFlrSF', 'OverallQual', 'TotalBsmtSF']
for _col in _NUMERIC_COLS_WITH_MISSING_VALUES:
    __simple_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    __train_dataset[_col] = __simple_imputer.fit_transform(__train_dataset[_col].values.reshape(-1,1))[:,0]
    if _col in __test_dataset:
        __test_dataset[_col] = __simple_imputer.transform(__test_dataset[_col].astype(__train_dataset[_col].dtypes).values.reshape(-1,1))[:,0]

## Remove Missing data in Categorical Columns

In the given input dataset there are <b>43 columns </b> with  missing data as follows:

Condition2, Fence, LandSlope, GarageCond, BsmtExposure, LotConfig, HeatingQC, SaleCondition, MSZoning, Alley, Exterior1st, Exterior2nd, Heating, HouseStyle, Utilities, BsmtCond, SaleType, BldgType, FireplaceQu, Street, BsmtFinType2, ExterQual, ExterCond, BsmtFinType1, Functional, GarageQual, CentralAir, LandContour, LotShape, GarageFinish, PoolQC, Foundation, RoofMatl, Neighborhood, GarageType, PavedDrive, KitchenQual, Condition1, Electrical, MiscFeature, MasVnrType, BsmtQual, RoofStyle

The following code removes the missing values.

In [9]:
# PREPROCESSING-2
from sklearn.impute import SimpleImputer
import numpy as np
_STRING_COLS_WITH_MISSING_VALUES = ['Condition2', 'Fence', 'LandSlope', 'GarageCond', 'BsmtExposure', 'LotConfig', 'HeatingQC', 'SaleCondition', 'MSZoning', 'Alley', 'Exterior1st', 'Exterior2nd', 'Heating', 'HouseStyle', 'Utilities', 'BsmtCond', 'SaleType', 'BldgType', 'FireplaceQu', 'Street', 'BsmtFinType2', 'ExterQual', 'ExterCond', 'BsmtFinType1', 'Functional', 'GarageQual', 'CentralAir', 'LandContour', 'LotShape', 'GarageFinish', 'PoolQC', 'Foundation', 'RoofMatl', 'Neighborhood', 'GarageType', 'PavedDrive', 'KitchenQual', 'Condition1', 'Electrical', 'MiscFeature', 'MasVnrType', 'BsmtQual', 'RoofStyle']
for _col in _STRING_COLS_WITH_MISSING_VALUES:
    __simple_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    __train_dataset[_col] = __simple_imputer.fit_transform(__train_dataset[_col].values.reshape(-1,1))[:,0]
    if _col in __test_dataset:
        __test_dataset[_col] = __simple_imputer.transform(__test_dataset[_col].astype(__train_dataset[_col].dtypes).values.reshape(-1,1))[:,0]

## Encode categorical features
Encode categorical features as an integer array.

In the given input dataset there are <b>43 columns </b> with string values as follows:

MSZoning, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, CentralAir, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, MiscFeature, SaleType, SaleCondition

The following code encode the selected columns.

In [10]:
# PREPROCESSING-3
from sklearn.preprocessing import OrdinalEncoder
_CATEGORICAL_COLS = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
_ohe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
__train_dataset[_CATEGORICAL_COLS] = pd.DataFrame(_ohe.fit_transform(__train_dataset[_CATEGORICAL_COLS]), columns=_CATEGORICAL_COLS)
__test_dataset[_CATEGORICAL_COLS] = pd.DataFrame(_ohe.transform(__test_dataset[_CATEGORICAL_COLS]), columns=_CATEGORICAL_COLS)

## Numeric to Scale

In the given input dataset there are <b>37 columns </b> with numeric values as follows where we can convert those values to scale through [log1p](https://numpy.org/doc/stable/reference/generated/numpy.log1p.html).

MSSubClass, LotFrontage, LotArea, OverallQual, OverallCond, YearBuilt, YearRemodAdd, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageYrBlt, GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal, MoSold, YrSold, SalePrice

In [11]:
# PREPROCESSING-4
import numpy as np
_NUMERIC_COLS_TO_SCALE = ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
for _col in _NUMERIC_COLS_TO_SCALE:
    __train_dataset[_col] = np.log1p(__train_dataset[_col])
    if _col in __test_dataset:
        __test_dataset[_col] = np.log1p(__test_dataset[_col])

### Target Column
The target column is the value which we need to predict.
Therefore, we need to detach the target columns in prediction.
Note that if we don't drop this fields, it will generate a model with high accuracy on training and worst accuracy on test (because the value in test dataset is Null).
Target column: "SalePrice"

In [12]:
# DETACH TARGET
__feature_train = __train_dataset.drop(['SalePrice'], axis=1)
__target_train =__train_dataset['SalePrice']
__feature_test = __test_dataset

# Training Model and Prediction
First, we will train a model based on preprocessed values of training data set.
Second, let's predict test values based on the trained model.

## CatBoostRegressor
We will use CatBoostRegressor which is a fast, scalable, high performance gradient boosting on decision trees library. Used for ranking, classification, regression and other ML tasks.
[CatBoostRegressor Refenece](https://catboost.ai/docs/installation/python-installation-method-pip-install)

In [13]:
# MODEL
import numpy as np
from catboost import CatBoostRegressor
__model = CatBoostRegressor()
__model.fit(__feature_train, __target_train)
__y_pred = __model.predict(__feature_test)

Learning rate set to 0.043466
0:	learn: 0.3877327	total: 55.4ms	remaining: 55.3s
1:	learn: 0.3771788	total: 59ms	remaining: 29.5s
2:	learn: 0.3674571	total: 62.6ms	remaining: 20.8s
3:	learn: 0.3573823	total: 66.8ms	remaining: 16.6s
4:	learn: 0.3487148	total: 70.4ms	remaining: 14s
5:	learn: 0.3390291	total: 73.8ms	remaining: 12.2s
6:	learn: 0.3300573	total: 77.5ms	remaining: 11s
7:	learn: 0.3216739	total: 81.3ms	remaining: 10.1s
8:	learn: 0.3140349	total: 84.9ms	remaining: 9.35s
9:	learn: 0.3067539	total: 88.6ms	remaining: 8.77s
10:	learn: 0.2990794	total: 92.6ms	remaining: 8.33s
11:	learn: 0.2921042	total: 96.2ms	remaining: 7.92s
12:	learn: 0.2852626	total: 101ms	remaining: 7.64s
13:	learn: 0.2789428	total: 107ms	remaining: 7.5s
14:	learn: 0.2727331	total: 111ms	remaining: 7.32s
15:	learn: 0.2671083	total: 118ms	remaining: 7.24s
16:	learn: 0.2611467	total: 127ms	remaining: 7.35s
17:	learn: 0.2561398	total: 139ms	remaining: 7.6s
18:	learn: 0.2510343	total: 152ms	remaining: 7.85s
19:	lea

In [14]:
# INVERSE TARGET
import numpy as np
if '__target_test' in locals():
    __target_test = np.expm1(__target_test)
__y_pred = np.expm1(__y_pred)

# Submission File
We have to maintain the target columns in submission.csv.

In [15]:
# SUBMISSION
submission = pd.DataFrame(columns=['Id'], data=__test_dataset_submission_columns)
submission['SalePrice'] = __y_pred
submission.to_csv("kaggle_submission.csv", index=False)